In [ ]:
%run "0_Workspace_setup.ipynb"

from ipywidgets import widgets
from IPython.display import display
import webbrowser
from NHM_helpers.map_template import make_var_map
from NHM_helpers.NHM_hydrofabric import make_hf_map_elements
from NHM_helpers.NHM_output_visualization import retrieve_hru_output_info
from ipywidgets import VBox

# temp fix for output plots below, plot_colors should prob be nested in fujnctions that need them
from NHM_helpers.output_plots import *
# from NHM_helpers.output_plots import(
# create_var_ts_for_poi_basin_df,
# make_plot_var_for_hrus_in_poi_basin,
# oopla,
# )
poi_id_sel = None

In [ ]:
(
    hru_gdf,
    hru_txt,
    hru_cal_level_txt,
    seg_gdf,
    seg_txt,
    nwis_gages_aoi,
    poi_df,
    gages_df,
    gages_txt,
    gages_txt_nb2,
    HW_basins_gdf,
    HW_basins,
) = make_hf_map_elements(
    NHM_dir,
    model_dir,
    GIS_format,
    param_filename,
    control_file_name,
    nwis_gages_file,
    gages_file,
    default_gages_file,
    nhru_params,
    nhru_nmonths_params,
    nwis_gage_nobs_min,
)

# Retrieve pywatershed output file information for mapping and plotting
plot_start_date, plot_end_date, year_list, output_var_list = retrieve_hru_output_info(
    out_dir,
    water_years,
)

con.print(
    f"{workspace_txt}\n",
    f"\n{gages_txt}{seg_txt}{hru_txt}",
    f"\n     {hru_cal_level_txt}",
)

#### &#x270D;<font color='green'>**Enter Information:**</font> **Select HRU **output variable** and a **year** to map..
> The default is set to **recharge**, **mean_annual** with no gage selected. If a gage number is selected here,
> the map will zoom to that location.

In [ ]:
# Create selection widgets
##### Varibale selection widget
output_var_sel = output_var_list[
    8
]  # Set a default value so that the notebook will run without selection

style_date = {"description_width": "initial"}

style_var = {"description_width": "initial"}

v = widgets.Dropdown(
    options=output_var_list,
    value=output_var_list[1],
    description="Output variable for map and plot:",
    layout=widgets.Layout(width="35%"),
    style=style_var,
)


def on_change(change):
    global output_var_sel, sel_flag
    if change["type"] == "change" and change["name"] == "value":
        output_var_sel = v.value
        # sel_flag = True


v.observe(on_change)
# display(v)

##### Year selection widget
list_of_years = year_list.copy()
list_of_years.append(
    "mean_annual"
)  # Append the mean annual so that the default will not be a year
sel_year = list_of_years[
    -1
]  # Set a default value so that the notebook will run without selection

yr = widgets.Dropdown(
    options=list_of_years,
    value=list_of_years[-1],
    description="Time step (year) for map:",
    layout=widgets.Layout(width="35%"),
    style=style_var,
)


def on_change(change):
    global sel_year  # Have to set the var as global so that it is carried outside of the fucntion to the notebook
    if change["type"] == "change" and change["name"] == "value":
        sel_year = yr.value


yr.observe(on_change)

# #################################################
v2 = widgets.Combobox(
    # value=poi_df.poi_id.tolist()[0],
    placeholder="(optional) Enter Gage ID here",
    options=poi_df.poi_id.tolist(),
    description="Plot Gage:",
    ensure_option=True,
    disabled=False,
    layout=widgets.Layout(width="35%"),
    style=style_var,
)


def on_change2(change):
    global poi_id_sel, fig
    if change["type"] == "change" and change["name"] == "value":
        poi_id_sel = v2.value


v2.observe(on_change2)

display(VBox([v, yr, v2]))

#### &#x1F6D1;If new selections are made above,
><font color='green'>**select this cell**</font>, then select <font color='green'>**Run Selected Cell and All Below**</font> from the Run menu in the toolbar.

## Make interactive map of selected output variable values in the NHM domain

In [ ]:
# Make map

# This is for testing only; can comment out in user version
if poi_id_sel is None:
    poi_id_sel = poi_df.poi_id.tolist()[0]


map_file = make_var_map(
    out_dir,
    output_var_sel,
    plot_start_date,
    plot_end_date,
    water_years,
    hru_gdf,
    poi_df,
    poi_id_sel,
    seg_gdf,
    nwis_gages_aoi,
    gages_df,
    html_maps_dir,
    param_filename,
    subbasin,
    year_list,
    sel_year,
    Folium_maps_dir,
    HW_basins_gdf,
    HW_basins,
)

map_file_str = f"{map_file}"
_ = webbrowser.open(map_file_str, new=2)

## Create an interactive plot to evaluate an NHM ouput variable for poi basin.
> A plot is created for the output variable selected in this notebook that shows a cummulative value for the selected poi basin at the annual, monthly, and daily time steps. Individual poi basin hru contributions can also be compared to the poi basin values. The plot is interactive and can be saved using the widgets in the upper right-hand corner of the plot.

In [ ]:
# This is for testing only; can comment out in user version
if poi_id_sel is None:
    poi_id_sel = poi_df.poi_id.tolist()[0]

fig_hru_sum_vars_for_poi_filename = make_plot_var_for_hrus_in_poi_basin(
    out_dir,
    param_filename,
    water_years,
    hru_gdf,
    poi_df,
    output_var_sel,
    poi_id_sel,
    plot_start_date,
    plot_end_date,
    plot_colors,
    subbasin,
    html_plots_dir,
)

## Create an interactive plot to evaluate all NHM ouput variables as average fluxes (cubic feet/sec) for a poi basin.
> A plot is created for all listed output variables that shows average fluxes (cubic feet/sec) for the selected poi basin at the annual, monthly, and daily time steps. Individual output variables can be added and removed. The plot is interactive and can be saved using the widgets in the upper right-hand corner of the plot.

In [ ]:
fig_var_fluxes_for_poi_filename = oopla(
    out_dir,
    param_filename,
    water_years,
    hru_gdf,
    poi_df,
    output_var_list,
    output_var_sel,
    poi_id_sel,
    plot_start_date,
    plot_end_date,
    plot_colors,
    var_colors_dict,
    leg_only_dict,
    subbasin,
    html_plots_dir,
)

# fig_var_fluxes_for_poi.show()